In [12]:
!pip -q install langchain tiktoken qdrant-client  pypdf langchainhub  langchain_community langchain-groq langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 9.9 MB/s eta 0:00:00


In [2]:
from langchain_groq import ChatGroq
from langchain import hub
import tiktoken
from langchain_community.vectorstores import Qdrant
from qdrant_client import QdrantClient
from google.colab import userdata
from qdrant_client import QdrantClient
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.prompts import PromptTemplate

In [3]:
qdrant_url = userdata.get('QDRANT_URL')
qdrant_api_key = userdata.get('QDRANT_API_KEY')
groq_api_key = userdata.get('GROQ_API_KEY')

In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/a_thousand_splendid_sun.pdf")

documents=loader.load()

In [9]:
documents[:5]

[Document(metadata={'source': '/content/a_thousand_splendid_sun.pdf', 'page': 0}, page_content='A THOUSAND SPLENDID SUNS             \n1 \n  \n \n \n \n \n \n \n \n \n \n \n \n \n \nTitle: A Thousand Splendid Suns  \nAuthor: Khaled Hosseini  \nEdition: IV  \nLanguage: English  \nDate first posted: July 2007  \nDate  recently updated: Jan 2008  \n \nThis eBook was produ ced by: Larry Zhao in Beijing  '),
 Document(metadata={'source': '/content/a_thousand_splendid_sun.pdf', 'page': 1}, page_content='A THOUSAND SPLENDID SUNS             \n2 \n  \nA THOUSAND  SPLENDID  SUNS \n \nKHALED HOSSEINI  \n \n \n \n \n \nA T HOUSAND  SPLENDID  SUNS \n \nRIVEKHEAD BOOKS  \n \nA member of Penguin Group (USA) Inc. New York  \n \n2007  \n \nFrom the Library of Nicolas Naprstek  \n \n \n \n \n '),
 Document(metadata={'source': '/content/a_thousand_splendid_sun.pdf', 'page': 2}, page_content='A THOUSAND SPLENDID SUNS             \n3 \n RIVERHEAD BOOKS  \n \nPublished by the Penguin Group  \nPenguin Group

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2500,chunk_overlap=300)
texts = text_splitter.split_documents(documents)

In [10]:
len(documents)

253

In [11]:
len(texts)

397

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
# dont run this code again n again, just run it once

qdrant=Qdrant.from_documents(
    texts,
    embedding = embeddings,
    url=qdrant_url,
    prefer_grpc=True,
    api_key=qdrant_api_key,
    collection_name="llm-app-new"
)

In [21]:
qdrant_client = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)

In [23]:
# to retrieve 5 results
retriever=qdrant.as_retriever(search_kwargs={"k": 5})

In [24]:
query = "Who was Marium?"
docs=retriever.get_relevant_documents(query)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [25]:
#for better print of the docs
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [26]:
pretty_print_docs(docs)

Document 1:

A THOUSAND SPLENDID SUNS             
21 
  
  At the stream, Mariam waited by the spot they had agreed on the day before. In the sky, a 
few gray, cauliflower shaped clouds drifted by. Jali l had taught her that gray clouds got 
their color by being so dense that their top parts absorbed the sunlight and cast their own 
shadow along the base.  That's what you see, Mariam jo,  he had said,  the dark in their 
underbelly.  
 
  Some time passed.  
 
  Mari am went back to the kolba  This time, she walked around the west facing periphery of 
the clearing so she wouldn't have to pass by Nana. She checked the clock. It was almost one 
o'clock.  
 
  He's a businessman,  Mariam thought.  Something has come up.  
 
  She we nt back to the stream and waited awhile longer. Blackbirds circled overhead, 
dipped into the grass somewhere. She watched a caterpillar inching along the foot of an 
immature thistle.  
 
  She waited until her legs were stiff. This time, she did not go back

In [ ]:
retriever=qdrant.as_retriever(search_kwargs={"k": 10})

prompt=PromptTemplate(

template="""
# Your role
You are a brilliant expert at understanding the intent of the questioner and the crux of the question, and providing the most optimal answer  from the docs to the questioner's needs from the documents you are given.


# Instruction
Your task is to answer the question  using the following pieces of retrieved context delimited by XML tags.

<retrieved context>
Retrieved Context:
{context}
</retrieved context>


# Constraint
1. Think deeply and multiple times about the user's question\nUser's question:\n{question}\nYou must understand the intent of their question and provide the most appropriate answer.
- Ask yourself why to understand the context of the question and why the questioner asked it, reflect on it, and provide an appropriate response based on what you understand.
2. Choose the most relevant content(the key content that directly relates to the question) from the retrieved context and use it to generate an answer.
3. Generate a concise, logical answer. When generating the answer, Do Not just list your selections, But rearrange them in context so that they become paragraphs with a natural flow.
4. When you don't have retrieved context for the question or If you have a retrieved documents, but their content is irrelevant to the question, you should answer 'I can't find the answer to that question in the material I have'.
5. Use five sentences maximum. Keep the answer concise but logical/natural/in-depth.
6. At the end of the response provide metadata provided in the relevant docs , For example:"Metadata: page: 19, source: /content/OCR_RSCA/Analyse docs JVB + mails et convention FOOT INNOVATION.pdf'. Return Just the page and source


# Question:
{question}""",
    input_variables=["context","question"]
)
groq_llm= ChatGroq(
    model="llma-3.1-70b-versatile",
    temperature=0,
    groq_api_key=groq_api_key
    max_retries=2,
)


def format_docs(docs):
    formatted_docs = []
    for doc in docs:
        # Format the metadata into a string
        metadata_str = ', '.join(f"{key}: {value}" for key, value in doc.metadata.items())

        # Combine page content with its metadata
        doc_str = f"{doc.page_content}\nMetadata: {metadata_str}"

        # Append to the list of formatted documents
        formatted_docs.append(doc_str)

    # Join all formatted documents with double newlines
    return "\n\n".join(formatted_docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | groq_llm
    | StrOutputParser()
)